In [33]:
import pickle
import sys
import os
import json
import pandas as pd
import ast
import numpy as np
# add utils to path
sys.path.append('/root/capsule/code/beh_ephys_analysis')
from utils.beh_functions import parseSessionID

In [34]:
# Load the CCF summary CSV file
anti_file_path = '/root/capsule/code/data_management/antidromic_unit_summary_manual_072725.csv'
anti_df = pd.read_csv(anti_file_path)
anti_df['found'] = 0
# load the combined pickle file
combined_pickle_path = '/root/capsule/scratch/combined/combine_unit_tbl/combined_unit_tbl.pkl'
with open(combined_pickle_path, 'rb') as f:
    combined_unit_tbl = pickle.load(f)

In [35]:
anti_df['session'] = anti_df['session'].ffill()

In [36]:
skipping_ani_ids = []

In [37]:
combined_unit_tbl['anti_resp'] = 0
for ind, row in anti_df.iterrows():
    session = row['session']
    unit = row['unit_id']
    anti = row['tier']
    proj = row['projection']
    aniID, date_obj, raw_id = parseSessionID(session)
    if aniID not in skipping_ani_ids:
        if combined_unit_tbl[(combined_unit_tbl['session'] == session) & (combined_unit_tbl['unit'] == unit)].empty:
            print(f"Skipping {session} {unit} as it does not exist in combined_unit_tbl")
            continue
        curr_optp_resp = combined_unit_tbl[(combined_unit_tbl['session'] == session) & (combined_unit_tbl['unit'] == unit)]['anti_resp'].values[0]
        combined_unit_tbl.loc[(combined_unit_tbl['session'] == session) & (combined_unit_tbl['unit'] == unit), 'anti_resp'] = max(curr_optp_resp, anti)
        combined_unit_tbl.loc[(combined_unit_tbl['session'] == session) & (combined_unit_tbl['unit'] == unit), 'proj'] = proj
        anti_df.at[ind, 'found'] = 1

In [38]:
# update the existing pickle file with the new CCF coordinates
with open(combined_pickle_path, 'wb') as f:
    pickle.dump(combined_unit_tbl, f)

In [39]:
anti_df.to_csv(anti_file_path, index=False)